# Seal Detection - Test & Visualization

シール検出のテストと可視化を行うノートブック

## 機能
- 画像/動画からの検出テスト
- 検出結果の可視化
- バッチ処理
- 出力動画の生成

In [ ]:
# ライブラリのインポート
import cv2
import numpy as np
import yaml
import time
from pathlib import Path
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Video
import ipywidgets as widgets

# matplotlibの設定
%matplotlib inline
plt.rcParams['figure.figsize'] = [14, 8]
plt.rcParams['figure.dpi'] = 100

# 検出器のインポート
import sys
sys.path.append('/workspace/scripts')

from color_shape_detector import ColorShapeDetector, Detection
print("✓ Libraries loaded")

## 1. 検出器の初期化

In [ ]:
# 設定ファイルのパス
CONFIG_PATH = '/workspace/scripts/detector_config.yaml'

# 検出器を初期化
detector = ColorShapeDetector(CONFIG_PATH)

# 現在の設定を表示
print("Current Configuration:")
print("=" * 50)
print(yaml.dump(detector.config, default_flow_style=False))

## 2. 画像からの検出

In [ ]:
def detect_and_show(image_path, detector, show_masks=True):
    """
    画像から検出して結果を表示
    """
    # 画像読み込み
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"✗ Cannot read: {image_path}")
        return None
    
    print(f"Image: {image_path}")
    print(f"Size: {frame.shape[1]}x{frame.shape[0]}")
    
    # 検出実行
    start_time = time.time()
    detections = detector.detect(frame)
    elapsed = time.time() - start_time
    
    print(f"Detection time: {elapsed*1000:.1f}ms")
    print(f"Found {len(detections)} objects")
    
    # 結果描画
    result = detector.draw_detections(frame, detections, draw_hull=True, draw_vertices=True)
    
    # 表示
    if show_masks:
        masks = detector.get_debug_masks(frame)
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        axes[0, 0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        axes[0, 0].set_title('Original')
        axes[0, 0].axis('off')
        
        axes[0, 1].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
        axes[0, 1].set_title(f'Detection Result ({len(detections)} objects)')
        axes[0, 1].axis('off')
        
        axes[1, 0].imshow(masks['blue'], cmap='Blues')
        axes[1, 0].set_title('Blue Mask')
        axes[1, 0].axis('off')
        
        axes[1, 1].imshow(masks['yellow'], cmap='YlOrBr')
        axes[1, 1].set_title('Yellow Mask')
        axes[1, 1].axis('off')
    else:
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        axes[0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        axes[0].set_title('Original')
        axes[0].axis('off')
        
        axes[1].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
        axes[1].set_title(f'Detection Result ({len(detections)} objects)')
        axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # 検出結果の詳細
    if detections:
        print("\n検出結果の詳細:")
        print("-" * 60)
        for i, det in enumerate(detections, 1):
            x, y, w, h = det.bbox
            print(f"{i}. {det.class_name}")
            print(f"   Position: ({x}, {y})")
            print(f"   Size: {w}x{h}")
            print(f"   Vertices: {det.vertices}")
            print(f"   Confidence: {det.confidence:.3f}")
    
    return detections

In [ ]:
# === 画像パスを指定して検出テスト ===
IMAGE_PATH = "/workspace/videos/sample.jpg"  # ← ここを変更

detections = detect_and_show(IMAGE_PATH, detector)

## 3. 動画からの検出

In [ ]:
def process_video_jupyter(
    video_path,
    output_path,
    detector,
    sample_interval=30,
    show_samples=5
):
    """
    動画を処理して出力動画を生成
    
    Args:
        video_path: 入力動画パス
        output_path: 出力動画パス
        detector: 検出器
        sample_interval: サンプル表示の間隔
        show_samples: 表示するサンプル数
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"✗ Cannot open: {video_path}")
        return
    
    # 動画情報
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video: {video_path}")
    print(f"Resolution: {width}x{height}")
    print(f"FPS: {fps}")
    print(f"Total frames: {total_frames}")
    print(f"Duration: {total_frames/fps:.1f}s")
    
    # 出力設定
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # 統計
    detection_counts = {'blue_triangle': 0, 'yellow_octagon': 0}
    sample_frames = []
    sample_results = []
    
    # 処理
    frame_count = 0
    total_time = 0
    
    for _ in tqdm(range(total_frames), desc="Processing"):
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # 検出
        start_time = time.time()
        detections = detector.detect(frame)
        elapsed = time.time() - start_time
        total_time += elapsed
        
        # 統計更新
        for det in detections:
            if det.class_name in detection_counts:
                detection_counts[det.class_name] += 1
        
        # 描画
        result = detector.draw_detections(frame, detections, draw_hull=True)
        
        # FPS表示
        current_fps = 1.0 / elapsed if elapsed > 0 else 0
        cv2.putText(result, f"FPS: {current_fps:.1f}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # 書き込み
        writer.write(result)
        
        # サンプル保存
        if frame_count % sample_interval == 0 and len(sample_frames) < show_samples:
            sample_frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            sample_results.append(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    
    cap.release()
    writer.release()
    
    # 結果表示
    print("\n" + "=" * 50)
    print("Processing Complete!")
    print("=" * 50)
    print(f"Processed frames: {frame_count}")
    print(f"Average FPS: {frame_count/total_time:.2f}")
    print(f"Output: {output_path}")
    print(f"\nDetection counts:")
    for name, count in detection_counts.items():
        print(f"  - {name}: {count}")
    
    # サンプル表示
    if sample_frames:
        print(f"\nSample frames (every {sample_interval} frames):")
        
        n_samples = len(sample_frames)
        fig, axes = plt.subplots(2, n_samples, figsize=(4*n_samples, 8))
        
        if n_samples == 1:
            axes = axes.reshape(2, 1)
        
        for i in range(n_samples):
            axes[0, i].imshow(sample_frames[i])
            axes[0, i].set_title(f'Frame {(i+1)*sample_interval}')
            axes[0, i].axis('off')
            
            axes[1, i].imshow(sample_results[i])
            axes[1, i].set_title('Detection')
            axes[1, i].axis('off')
        
        plt.tight_layout()
        plt.show()
    
    return detection_counts

In [ ]:
# === 動画処理（必要に応じてコメント解除）===
# VIDEO_INPUT = "/workspace/videos/input.mp4"
# VIDEO_OUTPUT = "/workspace/videos/output.mp4"

# process_video_jupyter(VIDEO_INPUT, VIDEO_OUTPUT, detector, sample_interval=100, show_samples=5)

## 4. インタラクティブなフレーム選択

In [ ]:
class VideoFrameExplorer:
    """動画フレームをインタラクティブに探索"""
    
    def __init__(self, video_path, detector):
        self.video_path = video_path
        self.detector = detector
        
        # 動画を開く
        self.cap = cv2.VideoCapture(video_path)
        if not self.cap.isOpened():
            raise ValueError(f"Cannot open: {video_path}")
        
        self.total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)
        
        # UI
        self.frame_slider = widgets.IntSlider(
            value=0, min=0, max=self.total_frames-1,
            description='Frame:',
            layout=widgets.Layout(width='80%')
        )
        self.frame_slider.observe(self._on_frame_change, names='value')
        
        self.output = widgets.Output()
        
        print(f"Video: {video_path}")
        print(f"Frames: {self.total_frames}, FPS: {self.fps}")
    
    def _on_frame_change(self, change):
        self._show_frame(change['new'])
    
    def _show_frame(self, frame_num):
        with self.output:
            clear_output = True
            from IPython.display import clear_output
            clear_output(wait=True)
            
            # フレーム読み込み
            self.cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            ret, frame = self.cap.read()
            
            if not ret:
                print(f"Cannot read frame {frame_num}")
                return
            
            # 検出
            detections = self.detector.detect(frame)
            result = self.detector.draw_detections(frame, detections, draw_hull=True, draw_vertices=True)
            
            # 表示
            fig, axes = plt.subplots(1, 2, figsize=(14, 6))
            
            axes[0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            axes[0].set_title(f'Frame {frame_num} (Time: {frame_num/self.fps:.2f}s)')
            axes[0].axis('off')
            
            axes[1].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
            axes[1].set_title(f'Detection ({len(detections)} objects)')
            axes[1].axis('off')
            
            plt.tight_layout()
            plt.show()
            
            # 検出情報
            if detections:
                for det in detections:
                    print(f"  • {det.class_name}: bbox={det.bbox}, conf={det.confidence:.2f}")
    
    def display(self):
        display(widgets.VBox([self.frame_slider, self.output]))
        self._show_frame(0)
    
    def close(self):
        self.cap.release()

In [ ]:
# === インタラクティブフレーム探索（必要に応じてコメント解除）===
# VIDEO_PATH = "/workspace/videos/input.mp4"
# explorer = VideoFrameExplorer(VIDEO_PATH, detector)
# explorer.display()

## 5. バッチ処理

In [ ]:
def batch_detect(image_dir, detector, output_dir=None):
    """
    ディレクトリ内の全画像を処理
    """
    image_dir = Path(image_dir)
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}
    
    image_files = [f for f in image_dir.iterdir() 
                   if f.suffix.lower() in image_extensions]
    
    if not image_files:
        print(f"No images found in {image_dir}")
        return
    
    print(f"Found {len(image_files)} images")
    
    # 出力ディレクトリ
    if output_dir:
        output_dir = Path(output_dir)
        output_dir.mkdir(parents=True, exist_ok=True)
    
    # 結果
    results = []
    
    for img_path in tqdm(image_files, desc="Processing"):
        frame = cv2.imread(str(img_path))
        if frame is None:
            continue
        
        detections = detector.detect(frame)
        
        results.append({
            'file': img_path.name,
            'detections': len(detections),
            'blue_triangle': sum(1 for d in detections if d.class_name == 'blue_triangle'),
            'yellow_octagon': sum(1 for d in detections if d.class_name == 'yellow_octagon'),
        })
        
        # 結果を保存
        if output_dir and detections:
            result_frame = detector.draw_detections(frame, detections, draw_hull=True)
            output_path = output_dir / f"detected_{img_path.name}"
            cv2.imwrite(str(output_path), result_frame)
    
    # 統計表示
    print("\n" + "=" * 50)
    print("Batch Processing Results")
    print("=" * 50)
    
    total_blue = sum(r['blue_triangle'] for r in results)
    total_yellow = sum(r['yellow_octagon'] for r in results)
    
    print(f"Total images: {len(results)}")
    print(f"Total blue_triangle: {total_blue}")
    print(f"Total yellow_octagon: {total_yellow}")
    
    # 検出があった画像のみ表示
    detected = [r for r in results if r['detections'] > 0]
    print(f"\nImages with detections: {len(detected)}")
    for r in detected[:10]:  # 最初の10件
        print(f"  {r['file']}: {r['blue_triangle']} blue, {r['yellow_octagon']} yellow")
    
    if len(detected) > 10:
        print(f"  ... and {len(detected) - 10} more")
    
    return results

In [ ]:
# === バッチ処理（必要に応じてコメント解除）===
# IMAGE_DIR = "/workspace/dataset/raw_images"
# OUTPUT_DIR = "/workspace/dataset/detected"
# results = batch_detect(IMAGE_DIR, detector, OUTPUT_DIR)

## 6. 結果画像の保存

In [ ]:
def save_detection_result(image_path, output_path, detector):
    """
    検出結果を画像として保存
    """
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"Cannot read: {image_path}")
        return
    
    detections = detector.detect(frame)
    result = detector.draw_detections(frame, detections, draw_hull=True, draw_vertices=True)
    
    cv2.imwrite(output_path, result)
    print(f"✓ Saved: {output_path}")
    print(f"  Detections: {len(detections)}")
    
    # 表示
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    plt.title(f'Detection Result ({len(detections)} objects)')
    plt.axis('off')
    plt.show()

In [ ]:
# === 結果を保存（必要に応じてコメント解除）===
# save_detection_result(
#     "/workspace/videos/sample.jpg",
#     "/workspace/videos/sample_result.jpg",
#     detector
# )

---

## クリーンアップ

In [ ]:
# インタラクティブ探索のリソース解放
# if 'explorer' in dir():
#     explorer.close()